### import

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import os.path
import os
#you will need to install library biopython
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein
from Bio.Phylo.PAML import codeml
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
%matplotlib inline

### Parse fasta 

In [2]:
# Download CDS for Human and Mouse from https://useast.ensembl.org/info/data/ftp/index.html
# for bactarial species from https://bacteria.ensembl.org/info/website/ftp/index.html

In [3]:
#species I will use 
list_sp=['H37Ra','Haemophilum'] # for first paper ['BW25113','Senterica'] , for second paper ['EcoliRel606','Senterica']  
# a dictinary with full names of CDS files
dist_sp=dist_sp={\
    'Senterica':'Salmonella_enterica.ASM78381v1.cds.all.fa',\
    'EcoliK12':'Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.cds.all.fa',\
    'EcoliRel606':'Escherichia_coli_b_str_rel606.ASM1798v1.cds.all.fa',\
    'BW25113':'Escherichia_coli_bw25113.ASM75055v1.cds.all.fa',\
    'Leprae':'Mycobacterium_leprae_br4923.ASM2668v1.cds.all.fa',\
    'Tuberc':'Mycobacterium_tuberculosis_h37rv.ASM19595v2.cds.all.fa',\
    'H37Ra': 'Mycobacterium_tuberculosis_h37ra.ASM1614v1.cds.all.fa',\
    'Haemophilum': 'Mycobacterium_haemophilum_gca_001021415.ASM102141v1.cds.all.fa'}

In [4]:
folder_home='DataPairOfSpecies/'+list_sp[0]+list_sp[1]+'/'

#create a directory where I will store info about this two species and thier alignmants
os.mkdir(folder_home) 

#create a directory for PAML results (the algorithm I will launch is called codeml)
os.mkdir(folder_home+'codeml')

In [5]:
#parse CDS fasta files ~2 min
dict_fasta={}
for sp in list_sp:
    print(sp)
    tmp_list_seq=[];tmp_list_id=[];tmp_list_descr=[];
    
    #read fasta file
    for record in SeqIO.parse("Data/cds_fasta/"+dist_sp[sp], "fasta"):
        tmp_list_seq.append(str(record.seq))
        tmp_list_id.append(record.id)
        tmp_list_descr.append(record.description)

    #save fasta in a dataframe
    dict_fasta[sp]=pd.DataFrame([[tmp_list_id[i],tmp_list_descr[i],tmp_list_seq[i]] for i in range(len(tmp_list_id))],\
            columns=['id','descr','seq'])
    
    #calculate length of sequence
    dict_fasta[sp]['len']=dict_fasta[sp]['seq'].apply(lambda x: len(x))
    #keep only those genes which nucleotyde sequence length %3. (this removes only a few genes)
    dict_fasta[sp]=dict_fasta[sp][dict_fasta[sp]['len'].apply(lambda x: x%3)==0]
    dict_fasta[sp]['len_aa']=dict_fasta[sp]['len'].apply(lambda x: x/3)
    
    #cut gene id from description field. 
    dict_fasta[sp]['id_gene']=dict_fasta[sp]['descr'].apply(lambda x: x.split('gene:')[1].split()[0])
    dict_fasta[sp]['id_gene']=dict_fasta[sp]['id_gene'].apply(lambda x: x.split('.')[0])
    #cut symbolic gene id from description field. 
    dict_fasta[sp]['id_symbol']=dict_fasta[sp]['descr'].\
    apply(lambda x: x.split('gene_symbol:')[1].split()[0] if 'gene_symbol:' in x else np.nan)
    
    #translate nuleotyde sequence into amino acid sequence 
    dict_fasta[sp]['seq_aa']=dict_fasta[sp]['seq'].apply(lambda x: str(Seq(x).translate()))

    #important step! Original file contains all isoforms for a gene. 
    #Here I sort by length and keep the longest isoform. 
    #(It is not ideal, maybe we can find somewhere information about "main" isoform)
    dict_fasta[sp]=dict_fasta[sp].sort_values(by='len',ascending=False).drop_duplicates('id_gene')
    


H37Ra
Haemophilum


In [6]:
#save CDS fasta for aa and nuc  
for sp in list_sp:
    text=''
    for id_transcript, seq_aa in dict_fasta[sp][['id','seq_aa']].values:
        text+='>'+id_transcript+'\n'+seq_aa[:-1]+'\n'

    fout=open(folder_home+'cds_fasta_aa_'+sp+'.fasta','w')
    fout.write(text)
    fout.close()
    
for sp in list_sp:
    text=''
    for id_transcript, seq_aa in dict_fasta[sp][['id','seq']].values:
        text+='>'+id_transcript+'\n'+seq_aa[:-1]+'\n'

    fout=open(folder_home+'cds_fasta_nuc_'+sp+'.fasta','w')
    fout.write(text)
    fout.close()

### BLAST with Diamond

In [7]:
#diamond is analogous software to BLAST, but works 10 times faster.
#I launch it from python. "!" in jupyter notebook makes the line to be execcuted as it was typed in the command line

In [8]:
#pairwise blast with diamond. ~10 min
os.chdir(folder_home)

for sp in list_sp:
    !diamond makedb --in cds_fasta_aa_{sp}.fasta --db db_{sp}
    
out_format='qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qseq_gapped sseq_gapped'
for sp1,sp2 in [list_sp,list_sp[::-1]]:
    !diamond blastp --query cds_fasta_aa_{sp1}.fasta --db db_{sp2}.dmnd --tantan-minMaskProb 1.0 --outfmt 6 {out_format} --out blast_{sp1}.vs.{sp2}.txt
os.chdir('../..')

diamond v2.0.2.140 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 8
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: cds_fasta_aa_H37Ra.fasta
Opening the database file...  [0.002s]
Loading sequences...  [0.027s]
Masking sequences...  [0.048s]
Writing sequences...  [0s]
Hashing sequences...  [0s]
Loading sequences...  [0s]
Writing trailer...  [0s]
Closing the input file...  [0.009s]
Closing the database file...  [0.001s]
Database hash = 20cd80090af06d2c66f20eaf10ec46bd
Processed 4034 sequences, 1340588 letters.
Total time = 0.09s
diamond v2.0.2.140 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 8
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: cds_fasta_aa_Haemophilum.fasta
Opening the database file..

### Parse blast results

In [9]:
#parse blast results

sp1_0=list_sp[0] #query
sp2_0=list_sp[1] #target
blast_dir={}

#read blast results for forward (Human->Mouse) and backward (Mouse->Human) directions
for sp1,sp2,direct in [[sp1_0,sp2_0,'fwd'],[sp2_0,sp1_0,'back']]:
    
    #read all blast results
    df=pd.read_csv(folder_home+'blast_'+sp1+'.vs.'+sp2+'.txt',sep='\t',header=None)
    #rename columns
    df=df.rename(columns={0:'id_'+sp1,1:'id_'+sp2,2:'ident',3:'len',4:'mism',5:'gaps',\
                      6:'qlo',7:'qhi',8:'tlo',9:'thi',10:'evalue',11:'bits',12:'qseq',13:'sseq'})
    
    #merge blast results with other information stored in the dict_fasta
    df=df.merge(dict_fasta[sp1][['id','id_gene','len_aa']].\
                rename(columns={'id':'id_'+sp1,'id_gene':'id_gene_'+sp1,'len_aa':'qlen'}),on='id_'+sp1,how='left')
    df=df.merge(dict_fasta[sp2][['id','id_gene','len_aa']].\
                rename(columns={'id':'id_'+sp2,'id_gene':'id_gene_'+sp2,'len_aa':'tlen'}),on='id_'+sp2,how='left')
    
    #keep only those results, where alignment length>30 and it covers more than 80% of protein length, 
    #and e-value is quite significant
    df=df[(df['len']>30)&\
          (df['qhi']-df['qlo']>0.8*df['qlen'])&(df['thi']-df['tlo']>0.8*df['tlen'])\
         &(df['evalue']<10**-6)]
    
    #keep only best hit for each gene.
    # it is a bit complicated, beacause I wanted to keep several hits it they are the same good.
    
    #first, I group lines by gene id and save the maximum identity for each gene in a df_max
    name_id='id_gene_'+sp1
    df_max=df[[name_id,'ident']].groupby(name_id).max() 
    df_max[name_id]=df_max.index
    df_max=df_max.set_index(df_max[name_id].values)
    df_max=df_max.rename(columns={'ident':'ident_max'}) 
    #second, I merge column "ident_max" to the main dataframe df
    df=df.merge(df_max, on=name_id,how='left')
    #third, I keep only those entries where ident==ident_max
    df=df[(df['ident']==df['ident_max'])]
  
    blast_dir[direct]=df
    print(len(df))
    
#merge forward and backward results of blast
blast=blast_dir['fwd'].merge(blast_dir['back'],on='id_'+sp2_0,how='inner',suffixes=['_'+sp1_0,'_'+sp2_0])

#keep only reciprocal hits
blast=blast[blast['id_gene_'+sp1_0+'_'+sp1_0]==blast['id_gene_'+sp1_0+'_'+sp2_0]]
blast=blast.drop_duplicates('id_gene_'+sp1_0+'_'+sp1_0)

print(sp2_0,len(blast_dir['fwd']),len(blast_dir['back']),len(blast_dir['fwd'].merge(blast_dir['back'],on='id_'+sp2_0,how='inner')),len(blast))

blast['id']=blast['id_gene_'+list_sp[0]+'_'+list_sp[0]]
blast=blast.set_index(blast['id'].values)

sp1=list_sp[0]
sp2=list_sp[1]
blast['id_gene_'+sp1]=blast['id_gene_'+sp1+'_'+sp1]
blast['id_gene_'+sp2]=blast['id_gene_'+sp2+'_'+sp1]
blast=blast.rename(columns={'id_'+sp1+'_'+sp1:'id_'+sp1})
#merge  blast results and sequences
for sp in list_sp:
    blast=blast.merge(dict_fasta[sp].rename(columns={'id':'id_'+sp,'seq_aa':'seq_aa_'+sp,'seq':'seq_'+sp})\
    [['id_'+sp,'seq_aa_'+sp,'seq_'+sp]],on='id_'+sp,how='left')
blast=blast.set_index(blast['id_gene_'+list_sp[0]].values)

2824
2862
Haemophilum 2824 2862 3119 2601


### Make nucl alignments by blast alignments

In [10]:
# Make nucl alignments by blast alignments. takes ~2 min   
sp1=list_sp[0]
sp2=list_sp[1]
for i in range(len(blast)):
    id_blast=blast.index[i]
    id1=blast.iloc[i]['id_gene_'+sp1]
    id2=blast.iloc[i]['id_gene_'+sp2]
    #whole amino acid sequence
    seq_aa1=blast.iloc[i]['seq_aa_'+sp1] 
    seq_aa2=blast.iloc[i]['seq_aa_'+sp2]
    #whole nucleotyde sequence
    seq_nuc1=blast.iloc[i]['seq_'+sp1] 
    seq_nuc2=blast.iloc[i]['seq_'+sp2]
    #sequnce of align part. With gaps inserted 
    seq_aa1_align=blast.iloc[i]['qseq_'+sp1] 
    seq_aa2_align=blast.iloc[i]['sseq_'+sp1]
    #srart and end of alignment
    start1=blast.iloc[i]['qlo_'+sp1]
    start2=blast.iloc[i]['tlo_'+sp1]
    end1=blast.iloc[i]['qhi_'+sp1]
    end2=blast.iloc[i]['thi_'+sp1]
    
    
    
    for sp,seq_aa,seq_nuc in [[sp1,seq_aa1_align, seq_nuc1[3*(start1-1):3*end1] ],\
                              [sp2,seq_aa2_align, seq_nuc2[3*(start2-1):3*end2] ]]:
        dna=''
        #I go through alignmnet and substitute each amino acid with corresponding nucleotyde codon or with '---' if it is gap 
        for aa in seq_aa:
            if aa=='-':
                dna+='---'
            elif aa=='*': 
                # "*" denotes stop-codons.
                dna+='---'
                seq_nuc=seq_nuc[3:]
                if len(seq_nuc)>0:
                    # this means that stop codon was found in the middle of the protein, such errors sometimes appear
                    # I print proteins where it happens. but so far ignore it.
                    print(i)
            else:
                dna+=seq_nuc[:3]
                seq_nuc=seq_nuc[3:]
               
        blast.at[id_blast,'seq_nuc_align_'+sp]=dna

In [11]:
# make text file with all alignments

sp1=list_sp[0]
sp2=list_sp[1]
text=''

for i in range(len(blast)):
    id1=blast.iloc[i]['id_gene_'+sp1]
    id2=blast.iloc[i]['id_gene_'+sp2]
    seq_nuc1=blast.iloc[i]['seq_nuc_align_'+sp1] 
    seq_nuc2=blast.iloc[i]['seq_nuc_align_'+sp2]
    
    
    text+='2\t'+str(len(seq_nuc1))+'\n>'+id1+'\n'+seq_nuc1+'\n'+'>'+id2+'\n'+seq_nuc2+'\n\n'

# write text file with all pairs of sequencies - the input for codeml
fout=open(folder_home+'codeml/all_pairs.fasta','w')
fout.write(text)
fout.close() 

### PAML to calculate dN and dS

In [12]:
fout=open(folder_home+'species.tree','w')
fout.write('(%s,%s);' %(list_sp[0],list_sp[1]))
fout.close()

In [14]:
# codeml from PAML to calculate dN, dS. ~10 min
# I launch PAML via biopython. I don't remember either PAML is fully installed if install biopython, or
# one need to install PAML modules separately. 
# "species.tree" file contains species tree, which is simple "(sp1,sp2);" in our case

# Because I constantly need to rerun. DELETE FOR DINARA
os.chdir("C:\\Users\\sysem\\Documents\\EvolutionRates\\Get_dN_dS")
# DELETE FOR DINARA


cml = codeml.Codeml(alignment = folder_home+'codeml/all_pairs.fasta',\
                    tree=folder_home+'species.tree',\
                    out_file = folder_home+'codeml/results.out',\
                    working_dir = folder_home+"codeml/")

# just keep these parameters as they are here
cml.set_options(
                noisy='9',\
                verbose='2',\
                runmode='-2',\
                seqtype='1',\
                model='0', \
                cleandata = '1',\
               CodonFreq = '2',\
               NSsites = '0',\
               fix_omega = '0',\
                ndata=len(blast)
)
# DELETE FOR DINARA TOO
res=cml.run(command="C:\\Users\\sysem\\Apps\\PAML\\paml4.9j\\bin\\codeml")



### Collect and save results

In [15]:
# Collect CODEML pairwise results
tmp=[]
with open(folder_home+'codeml/rst') as fin:
    for line in fin.readlines()[1+4::6]:
        tmp.append(map(lambda x: float(x),line.split()[2:7]))
df_pairs=pd.DataFrame(tmp,columns=['N', 'S', 'dN', 'dS', 'dN/dS'])

for col in ['N', 'S', 'dN', 'dS', 'dN/dS']:
    blast[col]=df_pairs[col].values

In [16]:
#save
blast.to_csv(folder_home+'blast_clock_all.txt',sep='\t',index=True)

sp1=list_sp[0]
sp2=list_sp[1]

blast.rename(columns={'id_gene_'+sp1+'_'+sp1:'id_'+sp1,'id_gene_'+sp2+'_'+sp1:'id_'+sp2,\
                      'id_'+sp2:'id_isoform_'+sp2,'id_'+sp1:'id_isoform_'+sp1,\
                     'ident_'+sp1:'ident','len_'+sp1:'len','mism_'+sp1:'mism','gaps_'+sp1:'gaps',\
                     'qlo_'+sp1:'qlo','qhi_'+sp1:'qhi','tlo_'+sp1:'tlo','thi_'+sp1:'thi',\
                     'evalue_'+sp1:'evalue','bits_'+sp1:'bits',\
                     'qlen_'+sp1:'qlen','tlen_'+sp1:'tlen'})\
[['id_'+sp1,'id_'+sp2,'dN','dS','dN/dS','N','S','id_isoform_'+sp1,'id_isoform_'+sp2,'ident','len','qlen','tlen','mism','gaps',\
 'qlo','qhi','tlo','thi','evalue','bits','seq_aa_'+sp1,'seq_aa_'+sp2,'ident_max_'+sp2]]\
.to_csv(folder_home+'blast_clock_short_seq.txt',sep='\t',index=True)

blast.rename(columns={'id_gene_'+sp1+'_'+sp1:'id_'+sp1,'id_gene_'+sp2+'_'+sp1:'id_'+sp2,\
                      'id_'+sp2:'id_isoform_'+sp2,'id_'+sp1:'id_isoform_'+sp1,\
                     'ident_'+sp1:'ident','len_'+sp1:'len','mism_'+sp1:'mism','gaps_'+sp1:'gaps',\
                     'qlo_'+sp1:'qlo','qhi_'+sp1:'qhi','tlo_'+sp1:'tlo','thi_'+sp1:'thi',\
                     'evalue_'+sp1:'evalue','bits_'+sp1:'bits',\
                     'qlen_'+sp1:'qlen','tlen_'+sp1:'tlen'})\
[['id_'+sp1,'id_'+sp2,'dN','dS','dN/dS','N','S','id_isoform_'+sp1,'id_isoform_'+sp2,'ident','len','qlen','tlen','mism','gaps',\
 'qlo','qhi','tlo','thi','evalue','bits','ident_max_'+sp2]]\
.to_csv(folder_home+'blast_clock_short.txt',sep='\t',index=True)

In [17]:
to_save=blast[['id','dN','dS']]
to_save=to_save.merge(dict_fasta[list_sp[0]][['id_gene','id_symbol']].rename(columns={'id_gene':'id'}),on='id',how='left')
to_save.loc[pd.isnull(to_save['id_symbol']),'id_symbol']=to_save.loc[pd.isnull(to_save['id_symbol']),'id']

In [18]:
to_save.to_csv(folder_home+'EvRate.txt',sep='\t',index=False)